# Import Library 

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy as sp
from scipy import stats
%matplotlib inline

In [3]:
pd.options.display.max_colwidth = 100

# Import Raw process data

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Wed Jul 24 17:09:34 2019
@author: tai
"""
import pandas as pd
from datetime import datetime 
import os

def get_event_log_by_file(m,u,d):
    corePath = '/home/tai/Python_data_analytics/einstein/einstein-anon/anon/activity/'
    path = corePath + m + '/'
    path += u + '/'
    path += d 
    """+ '-activity.log' """
    if os.path.exists(path):
        f = open(path,'r')
        temp = []
        for i in f:
            split = i.split(' ',6)
            if len(split)>5:
                if split[0] != ';;' and split[5] != 'upload' and split[5] != 'is-lab-exam:':
                    temp.append(split)
                
        data = pd.DataFrame(temp)
        data.columns = ["date", "time", "moduleCode", "org:resource", "case", "concept:name", "description"]        
        columns = ["case:concept:name", "concept:instance", "concept:name","time:timestamp", "org:resource", "lifecycle:transition","description"]
        data1 = pd.DataFrame(columns = columns)
        data1["case:concept:name"] = data["date"].apply(str) + '-' + data["org:resource"].apply(str) + '-' + data["case"]
        data1["concept:instance"] = data["concept:name"]
        data1["time:timestamp"] = data["date"].apply(str) + ' ' + data['time'].apply(str)
        data1["lifecycle:transition"] = 'complete'
        data1["org:resource"] = data["org:resource"]
        data1["concept:name"] = data["concept:name"]
        data1["description"] = data["description"]
        data1['time:timestamp'] = pd.to_datetime(data1.time:timestamp)
        """
        for index, row in data1.iterrows():             
            try:
                row['time:timestamp'] = datetime.strptime(row['time:timestamp'], '%Y-%m-%d %H:%M:%S')
            except:
                pass      
        """                
        return data1
    else:
        return []
    
    
def get_eventlogs_by_condition(module=[],user=[],date=[]):
    corePath = path = '/home/tai/Python_data_analytics/einstein/einstein-anon/anon/activity/'
    if len(module) == 0:
        path = corePath
        module = os.listdir(path)
    module = list(dict.fromkeys(module)) 
    
    if len(user) == 0:
        for m in module:
              path = corePath + m + '/'
              users = os.listdir(path)
              for u in users:
                  user.append(u)        
    user = list(dict.fromkeys(user))
    if len(date) == 0:
        for m in module:
            for u in user:
                path = corePath + m + '/' + u + '/'                
                dates = os.listdir(path)
                for d in dates:
                    date.append(d)    
    date = list(dict.fromkeys(date))
       
    logs = pd.DataFrame()
    for m in module:        
        for u in user:
            for d in date:                
                log = get_event_log_by_file(m,u,d)                
                if len(log) > 0:                    
                    logs = pd.concat([logs,log])
    
    return logs
def buildActivityUploadData(activity, allActivities, sourceData):
    actUpload = []
    for i in activity:
        act = allActivities.loc[allActivities['concept:name'] == i]
        act = act.groupby('org:resource').count()
        act.drop(['case:concept:name','concept:instance','time:timestamp','lifecycle:transition','description'], axis=1, inplace=True)
        columnName = i+'count'
        act.columns = [columnName]
        if len(actUpload) > 0:
            actUpload = actUpload.merge(act, left_on=data1.index, right_on=act.index)
        else:
            actUpload = data1.merge(act, left_on=data1.index, right_on=act.index)
        actUpload = actUpload.set_index('key_0')
    return actUpload
def pearsonGeneration(field,columns,data):
    array = []
    for i in field:   
        j = [i]
        j.extend(list(stats.pearsonr(activityUpload1[columns], activityUpload1[i+'count'])))
        array.append(j)
    result = pd.DataFrame(array,columns=['activity','correlation','p-value'])
    return result
""" some other things 
from os import listdir
from os.path import isfile, join
path = '/home/tai/Python_data_analytics/einstein/einstein-anon/anon/activity/'
a = os.listdir(path)
"""

module = ['ca116']
user = [] #get all users
date = [] #get all dates


log1 = get_eventlogs_by_condition(module,user,date=date)

log1.to_csv("activity1.csv",index=False) 

# Import assessment result data

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Mon Jul 29 11:46:49 2019
@author: tai
"""
import os
import json
import pandas as pd
import re
from datetime import datetime 

def getUploadReportContent(m,u,d,file):
    corePath = '/home/tai/Python_data_analytics/einstein/einstein-anon/anon/uploads/'
    path = corePath + m + '/'
    path += u + '/'
    path += d + '/'
    path += file
    if os.path.exists(path):
        f = open(path,'r')
        content1 = f.read()
        j = json.loads(content1)
        f.close()
        columns = ["date","module","user","task","language","correct","failed","passed","version","timeout","extension","ip"]
        content = []
        for index, c in enumerate(columns,start=0):
            content.append(j[c])
        
        return content
    else:
        return []

def extractUploadsData(module = [],user=[],date=[]):
    corePath = path = '/home/tai/Python_data_analytics/einstein/einstein-anon/anon/uploads/'
    if len(module) == 0:
        path = corePath
        if os.path.exists(path):
            module = os.listdir(path)
    module = list(dict.fromkeys(module)) 
    
    if len(user) == 0:
        for m in module:
              path = corePath + m + '/'
              if os.path.exists(path):
                  users = os.listdir(path)
                  for u in users:
                      user.append(u)        
    user = list(dict.fromkeys(user))

    if len(date) == 0:
        for m in module:
            for u in user:
                path = corePath + m + '/' + u + '/'
                if os.path.exists(path):                
                    dates = os.listdir(path)
                    for d in dates:
                        date.append(d)    
    date = list(dict.fromkeys(date))
    
    columns = ["date","module","user","task","language","correct","failed","passed","version","timeout","extension","ip"]
    
    resultArray = []
    for m in module:
        for u in user:
            for d in date:
                path = corePath + m + '/' + u + '/' + d + '/'
                if os.path.exists(path):
                    files = os.listdir(path)
                    for f in files:                        
                        if re.search(".report.(20|19)\d{2}-[0-1][0-9]-[0-3][0-9]-[0-2][0-9]:[0-6][0-9]:[0-6][0-9].json$",f):
                            temp = getUploadReportContent(m,u,d,f)
                            resultArray.append(temp)
    
    result = pd.DataFrame(resultArray,columns=columns)
    result['date'] = pd.to_datetime(result.date)
    return result  
   
result = extractUploadsData(module = ['ca116'])   


result.to_csv("uploads1.csv",index=False)    

"""
corePath = '/home/tai/Python_data_analytics/einstein/einstein-anon/anon/uploads/'
m = 'ca116'
u = 'u-00b8c129ae34eedef4311450577776a7a7f94c81'
d = '2018-09-27'
file = '.sweets-2.py.report.json'
path = corePath + m + '/'
path += u + '/'
path += d + '/'
path += file
print(path)
if os.path.exists(path):
    f = open(path,'r')
    content = f.read()
    j = json.loads(content)   
    print(j["results"][0]["correct"])
    print(len(j))
a = ".small-primes.py.report.2019-10-04-14:56:44.json"
b = re.findall(".report.(20|19)\d{2}-[0-1][0-9]-[0-3][0-9]-[0-2][0-9]:[0-6][0-9]:[0-6][0-9].json$",a)
print(b)
"""        

# Event log construction

In [ ]:
#
activity = pd.read_csv('~/Python_data_analytics/einstein/analysis/activity1.csv', sep=",")
activity = activity.drop(activity.index[140608])

In [ ]:
activity = activity.sort_values(by=['case:concept:name','time:timestamp'])

In [ ]:
activity['case'] = activity['case:concept:name']
activity['activity'] = activity['concept:instance']
eventLog = activity.set_index(['case', 'activity'])

In [ ]:
#eventLog = eventLog.loc[(eventLog['time:timestamp'] >= '2018-09-20') and eventLog['time:timestamp'] <= '2018-12-31')]
eventLog.sort_values(by=['case:concept:name','time:timestamp'])
#eventLog.to_csv('eventLog')
eventLog

In [ ]:
eventLog = pd.read_csv('eventLog.csv')

# Transition data matrix construction

In [ ]:
originalElements = ['load','scroll','blur','focus','unload','hashchange','click-0','selection','click-2',
                    'click-1','click-3']

columns = []
columns.append('case')
columns.append('startDate')
columns.append('endDate')
columns.append('user')

for i in originalElements:
    for j in originalElements:
        txt = i + '-' + j
        columns.append(txt)
columns = list(dict.fromkeys(columns))

In [ ]:
allRow = []
for index,row in eventLog.groupby(level=0):
    newRow = {}
    splitIndex = index.split("-")

    newRow['user'] = splitIndex[3]+'-'+splitIndex[4]
    newRow['startDate'] = row['time:timestamp'][0]
    newRow['endDate'] = row['time:timestamp'][len(row)-1]
    newRow['case'] = index
             
    for i in range(len(row['concept:instance'])-1):
        key = row['concept:instance'][i]+'-'+row['concept:instance'][i+1]
        if key in columns:
            if key not in newRow:
                newRow[key] = 1
            else:
                newRow[key] = newRow[key] + 1               
    allRow.append(newRow)

In [ ]:
activityVariance = pd.DataFrame(allRow,columns=columns)

In [ ]:
activityVariance = pd.read_csv('activityVariance2.csv')